In [1]:
import pandas as pd

import json
import glob
import os
import re

from shared import *

In [2]:
base = "../exp/"
paths = glob.glob(base + "epsilon/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'epsilon': run['epsilon'],
    'graph': path_to_graph(run['args'][1]),
} for run in data for algo in run['algo_runs']])
queries['nontrivial'] = queries['num_iterations'] > 1

In [4]:
queries.groupby(['epsilon', 'algo']).mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms', 'failed', 'nontrivial']]

length_increase_percent  num_iterations  \
epsilon algo                                                                 
0.01    iterative_detour_blocking                 0.716056           2.257   
        iterative_path_blocking                   0.340055          37.157   
0.05    iterative_detour_blocking                 0.467466           2.539   
        iterative_path_blocking                   0.280888          23.763   
0.10    iterative_detour_blocking                 0.330445           2.147   
        iterative_path_blocking                   0.231647          16.845   
0.20    iterative_detour_blocking                 0.256649           1.808   
        iterative_path_blocking                   0.201931          10.982   
0.50    iterative_detour_blocking                 0.170441           1.723   
        iterative_path_blocking                   0.147971           6.540   
1.00    iterative_detour_blocking                 0.120649           1.418   
        iterative_path_blocking                   0.102815           4.146   

                                   num_forbidden_paths  \
epsilon algo                                             
0.01    iterative_detour_blocking                4.822   
        iterative_path_blocking                107.684   
0.05    iterative_detour_blocking                4.426   
        iterative_path_blocking                 73.898   
0.10    iterative_detour_blocking                2.188   
        iterative_path_blocking                 54.351   
0.20    iterative_detour_blocking                1.440   
        iterative_path_blocking                 39.724   
0.50    iterative_detour_blocking                1.038   
        iterative_path_blocking                 25.035   
1.00    iterative_detour_blocking                0.594   
        iterative_path_blocking                 11.215   

                                   total_exploration_time_ms  \
epsilon algo                                                   
0.01    iterative_detour_blocking                  55.721072   
        iterative_path_blocking                    17.927375   
0.05    iterative_detour_blocking                  15.031331   
        iterative_path_blocking                     8.716707   
0.10    iterative_detour_blocking                   0.881003   
        iterative_path_blocking                     6.439434   
0.20    iterative_detour_blocking                   0.799085   
        iterative_path_blocking                     4.991557   
0.50    iterative_detour_blocking                   0.728252   
        iterative_path_blocking                     3.331414   
1.00    iterative_detour_blocking                   0.800571   
        iterative_path_blocking                     1.598925   

                                   total_ubs_time_ms  running_time_ms  failed  \
epsilon algo                                                                    
0.01    iterative_detour_blocking           5.223253        60.988437   0.006   
        iterative_path_blocking             6.171487        25.905175   0.102   
0.05    iterative_detour_blocking           5.090908        20.169559   0.005   
        iterative_path_blocking             4.924782        15.070991   0.056   
0.10    iterative_detour_blocking           3.407579         4.327785   0.003   
        iterative_path_blocking             4.323858        11.978743   0.040   
0.20    iterative_detour_blocking           2.805737         3.643991   0.002   
        iterative_path_blocking             3.305874         8.933532   0.022   
0.50    iterative_detour_blocking           2.467602         3.226342   0.002   
        iterative_path_blocking             2.601671         6.421596   0.012   
1.00    iterative_detour_blocking           1.724170         2.550635   0.001   
        iterative_path_blocking             2.327040         4.653761   0.006   

                                   nontrivial  
epsilon algo                                   
0

In [5]:
queries.query("~failed").groupby(['epsilon', 'algo']) \
    .mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms']] \
    .join(queries.groupby(['epsilon', 'algo']).mean()[['failed']])

length_increase_percent  num_iterations  \
epsilon algo                                                                 
0.01    iterative_detour_blocking                 0.560004        1.694165   
        iterative_path_blocking                   0.246349       12.867483   
0.05    iterative_detour_blocking                 0.388095        1.475377   
        iterative_path_blocking                   0.226360       10.282839   
0.10    iterative_detour_blocking                 0.331058        1.398195   
        iterative_path_blocking                   0.178666        7.088542   
0.20    iterative_detour_blocking                 0.257067        1.308617   
        iterative_path_blocking                   0.162082        5.582822   
0.50    iterative_detour_blocking                 0.170687        1.223447   
        iterative_path_blocking                   0.123570        3.570850   
1.00    iterative_detour_blocking                 0.120764        1.168168   
        iterative_path_blocking                   0.091997        2.655936   

                                   num_forbidden_paths  \
epsilon algo                                             
0.01    iterative_detour_blocking             3.204225   
        iterative_path_blocking              38.868597   
0.05    iterative_detour_blocking             1.730653   
        iterative_path_blocking              33.445975   
0.10    iterative_detour_blocking             1.427282   
        iterative_path_blocking              20.910417   
0.20    iterative_detour_blocking             0.939880   
        iterative_path_blocking              16.634969   
0.50    iterative_detour_blocking             0.537074   
        iterative_path_blocking              11.359312   
1.00    iterative_detour_blocking             0.343343   
        iterative_path_blocking               5.783702   

                                   total_exploration_time_ms  \
epsilon algo                                                   
0.01    iterative_detour_blocking                   0.899431   
        iterative_path_blocking                     4.217909   
0.05    iterative_detour_blocking                   0.837429   
        iterative_path_blocking                     3.661849   
0.10    iterative_detour_blocking                   0.836279   
        iterative_path_blocking                     2.811862   
0.20    iterative_detour_blocking                   0.777241   
        iterative_path_blocking                     2.930040   
0.50    iterative_detour_blocking                   0.707462   
        iterative_path_blocking                     1.944744   
1.00    iterative_detour_blocking                   0.788783   
        iterative_path_blocking                     1.195205   

                                   total_ubs_time_ms  running_time_ms  failed  
epsilon algo                                                                   
0.01    iterative_detour_blocking           3.150843         4.083773   0.006  
        iterative_path_blocking             2.543626         7.682042   0.102  
0.05    iterative_detour_blocking           2.291542         3.157566   0.005  
        iterative_path_blocking             2.593803         7.106448   0.056  
0.10    iterative_detour_blocking           2.115556         2.978257   0.003  
        iterative_path_blocking             2.260602         5.832500   0.040  
0.20    iterative_detour_blocking           1.883002         2.690705   0.002  
        iterative_path_blocking             2.009107         5.383889   0.022  
0.50    iterative_detour_blocking           1.543703         2.273418   0.002  
        iterative_path_blocking             1.742222         4.060571   0.012  
1.00    iterative_detour_blocking           1.563199         2.373572   0.001  
        iterative_path_blocking             1.991742         3.804074   0.006

In [22]:
table = queries.query("~failed").groupby(['epsilon', 'algo']) \
    .mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms']] \
    .join(queries.groupby(['epsilon', 'algo']).mean()[['failed']])

table['failed'] *= 100
cols1 = ['total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms', 'num_forbidden_paths']
cols2 = ['length_increase_percent', 'num_iterations']
table[cols1] = table[cols1].round(1)
table[cols2] = table[cols2].round(2)
table = table.rename(index={ 'iterative_detour_blocking': 'IDB', 'iterative_path_blocking': 'IPB' })

lines = table.rename(index=lambda x: R"\multirow{2}{*}{" + "{:.2f}".format(x) +R"}", level=0).to_latex(escape=False, column_format='ccrrrrrrr').split("\n")
lines = lines[:2] + [
  R"            & & Increase & Iterations & Blocked & \multicolumn{3}{c}{Running time [ms]} & Failed \\ \cmidrule(lr){6-8}",
  R" $\epsilon$ & &   $[\%]$ &            &   Paths & A* & UBS & Total                      & $[\%]$ \\"
] + lines[4:]
for l in range(6, 16, 2):
    lines[l] += '[2pt]'
output = add_latex_big_number_spaces("\n".join(lines) + "\n")
print(output)

table

\begin{tabular}{ccrrrrrrr}
\toprule
            & & Increase & Iterations & Blocked & \multicolumn{3}{c}{Running time [ms]} & Failed \\ \cmidrule(lr){6-8}
 $\epsilon$ & &   $[\%]$ &            &   Paths & A* & UBS & Total                      & $[\%]$ \\
\midrule
\multirow{2}{*}{0.01} & IDB &                     0.56 &            1.69 &                  3.2 &                        0.9 &                3.2 &              4.1 &     0.6 \\
                      & IPB &                     0.25 &           12.87 &                 38.9 &                        4.2 &                2.5 &              7.7 &    10.2 \\[2pt]
\multirow{2}{*}{0.05} & IDB &                     0.39 &            1.48 &                  1.7 &                        0.8 &                2.3 &              3.2 &     0.5 \\
                      & IPB &                     0.23 &           10.28 &                 33.4 &                        3.7 &                2.6 &              7.1 &     5.6 \\[2pt]
\multirow{2}{*

length_increase_percent  num_iterations  num_forbidden_paths  \
epsilon algo                                                                 
0.01    IDB                      0.56            1.69                  3.2   
        IPB                      0.25           12.87                 38.9   
0.05    IDB                      0.39            1.48                  1.7   
        IPB                      0.23           10.28                 33.4   
0.10    IDB                      0.33            1.40                  1.4   
        IPB                      0.18            7.09                 20.9   
0.20    IDB                      0.26            1.31                  0.9   
        IPB                      0.16            5.58                 16.6   
0.50    IDB                      0.17            1.22                  0.5   
        IPB                      0.12            3.57                 11.4   
1.00    IDB                      0.12            1.17                  0.3   
        IPB                      0.09            2.66                  5.8   

              total_exploration_time_ms  total_ubs_time_ms  running_time_ms  \
epsilon algo                                                                  
0.01    IDB                         0.9                3.2              4.1   
        IPB                         4.2                2.5              7.7   
0.05    IDB                         0.8                2.3              3.2   
        IPB                         3.7                2.6              7.1   
0.10    IDB                         0.8                2.1              3.0   
        IPB                         2.8                2.3              5.8   
0.20    IDB                         0.8                1.9              2.7   
        IPB                         2.9                2.0              5.4   
0.50    IDB                         0.7                1.5              2.3   
        IPB                         1.9                1.7              4.1   
1.00    IDB                         0.8                1.6              2.4   
        IPB                         1.2                2.0              3.8   

              failed  
epsilon algo          
0.01    IDB      0.6  
        IPB     10.2  
0.05    IDB      0.5  
        IPB      5.6  
0.10    IDB      0.3  
        IPB      4.0  
0.20    IDB      0.2  
        IPB      2.2  
0.50    IDB      0.2  
        IPB      1.2  
1.00    IDB      0.1  
        IPB      0.6